In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
#load movies dataset,check and remove NULL(since less than 1%)
movies = pd.read_csv('C:/Users/impca/Downloads/ml-20m/movies.csv')
movies.isnull().sum()
movies.dropna(inplace=True)

In [3]:
#seperating columns
movies["genres"]= movies["genres"].str.split("|") 
movies['year']= movies.title.str.extract("\((\d{4})\)")


C:\Users\impca\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# Assign genres to each movie.
genres = movies['genres']
genres = pd.get_dummies(genres.apply(pd.Series).stack()).sum(level=0)

In [5]:
movies['year']= pd.to_numeric(movies['year'])

In [6]:
#Normalize the Year of Movie Release so that 1990-later is "New", 1970s/1980s is "Medium" and anything in 1960s or before is "Old"
def f(x):
    if x['year'] >= 1990: return 'New'
    elif x['year'] < 1970:return 'Old'
    else: return 'Medium'
movies['year'] = movies.apply(f,axis=1)
movies['year'] = pd.factorize(movies['year'])[0]


In [7]:
#concat
movies = pd.concat([movies, genres], axis=1)


In [8]:
#load ratings dataset,check and remove NULL(no action)
ratings = pd.read_csv('C:/Users/impca/Downloads/ml-20m/ratings.csv')
ratings.isnull().sum()
ratings.dropna(inplace=True)

In [9]:
# Compute the average rating for each movie (pivot) and find average
table = pd.pivot_table(ratings, values='rating', index=[ 'movieId'], aggfunc=np.mean)


In [10]:
table.reset_index( inplace=True)


In [11]:
movies = movies.drop(['title','genres'],axis=1)

In [12]:
all_data = pd.merge(movies, table, 'inner', on = ["movieId"] )


In [13]:
all_data.head()

,movieId,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
0,1,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,3.921240
1,2,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.211977
2,3,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,3.151040
3,4,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,2.861393
4,5,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3.064592


In [14]:
#Sort the movies by their average rating
all_data = all_data.sort_values(by = ['rating']) 

In [15]:
# Find the median of average rating
med = all_data[['rating']]
med.median()


rating    3.235294
dtype: float64

In [16]:
#Pick top 20%, bottom 20% and middle 20% of the movies. You can call them "Best", "Worst and "OK" respectively. 
#naive approach
dfs = np.split(all_data, [5349,10698,16047,21396], axis=0)

In [17]:
bad = dfs[0]
Ok = dfs[2]
good = dfs[4]

In [18]:
bad = bad.assign(rating = 'bad')
Ok = Ok.assign(rating = 'OK')
good = good.assign(rating = 'good')


In [19]:
data = pd.concat([bad,Ok,good],axis = 0)
data['rating'] = pd.factorize(data['rating'])[0]


In [20]:
data.head()

,movieId,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
24647,117630,0,0,1,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
18913,94327,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26248,128675,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
20718,101728,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
21273,103769,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [21]:
X = data.iloc[:,1:22]


In [22]:
y = data[['rating']]

In [23]:
#Run a ML algorithm to show if/how a movie's Genre and/or Age determines its rating ("Best", "Worst", "OK").
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)

C:\Users\impca\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
from sklearn.linear_model import LogisticRegression 

In [25]:
logreg = LogisticRegression()

In [26]:
logreg.fit(X_train,y_train)

C:\Users\impca\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
yhat = logreg.predict(X_test)

In [28]:
from sklearn import metrics
print (metrics.accuracy_score(y_test,yhat))


0.4927295388450353


The base accuracy for the model is 33.3% i.e if all values were predicted to be of one class the accuracy would be 33.3%.Therefore the 49.2% accuracy obtained with Logistic Regression is a bad result.It can be thus, concluded that the prediction based on years and genres is not a great success